# 00 - Data Collection Notebook
The intent of this notebook is to acquire economic data, including but not limited to, industry and employment information at the state level for all 50 states, and Washington D.C. Once the data is compiled and transformed for each use case, the data is then exported into either the `source_data` or `state_employment` directories. 

*Some of the scraping methods used in this notebook were referenced from the following lecture: https://git.generalassemb.ly/DSIR-0124/lesson-webscraping/blob/master/intro-to-web-scraping-spiders-with-scrapy.ipynb*

#### Imports

In [1]:
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import os

In [2]:
np.random.seed(42)

### Individual State Scraper 
In this scraper, we extract one link per state rom the U.S. Bureau of Labor Statistics website found in the `url` variable below. As seen within this scraper, we chose to remove the U.S. territories of Guam, Puerto Rico, and the Virgin Islands for our analysis. 

Using BeautifulSoup, this scraper returns a Pandas DataFrame of the State name and associated URL.

In [3]:
url = 'https://www.bls.gov/eag'

response = requests.get(url)

html = response.text

soup = BeautifulSoup(html, 'lxml')

all_h4 = soup.find_all('h4')
all_h4

state_link_list = []

for element in all_h4:
    result = {}
    
    a_href = element.find('a')
    if a_href:
        result['title'] = a_href.text
        result['link'] = 'https://www.bls.gov/' + a_href['href'].strip().lstrip('/')
    
    state_link_list.append(result)
    
state_link_list

state_links = pd.DataFrame(state_link_list)
state_links = state_links.set_index('title')
state_links = state_links.drop(['Guam', 'Puerto Rico', 'Virgin Islands'])
state_links

,link
title,
Alabama,https://www.bls.gov/regions/southeast/alabama....
Alaska,https://www.bls.gov/regions/west/alaska.htm#eag
Arizona,https://www.bls.gov/regions/west/arizona.htm#eag
Arkansas,https://www.bls.gov/regions/southwest/arkansas...
California,https://www.bls.gov/regions/west/california.ht...
Colorado,https://www.bls.gov/regions/mountain-plains/co...
Connecticut,https://www.bls.gov/regions/new-england/connec...
Delaware,https://www.bls.gov/regions/mid-atlantic/delaw...
District of Columbia,https://www.bls.gov/regions/mid-atlantic/distr...


### All State Links Scraper 
In this scraper, we extract the necessary child links from the aforementioned DataFrame, using BeautifulSoup, and then save them into a Dictionary.

In [4]:
state_backdata = {}

for i in range(len(state_links)):
    state_url = state_links.iloc[i][0]
#     state_url
    state_name = state_links.index[i]
    
    loop_response = requests.get(state_url)

    loop_html = loop_response.text

    loop_soup = BeautifulSoup(loop_html, 'lxml')

    loop_table = loop_soup.find('table', {'class': 'regular'})

    loop_backdata_list = []

    loop_backdata_links = []
    
    for row in loop_table.find_all('tr')[1:]:

        loop_backdata_list.append(row)

    for item in loop_backdata_list:
        item = str(item)
        if 'https' in item:
            links= re.findall("(https:\S+)", item)
            loop_backdata_links.append(links[0])

    loop_final_list = [x for i, x in enumerate(loop_backdata_links) if i in [0,4,6,8,10,12,14,16,18,20,22,24,26,28]]
    state_backdata[state_name] = loop_final_list
    print(f'{state_name} links added to dictionary')
    time.sleep(1)

Alabama links added to dictionary
Alaska links added to dictionary
Arizona links added to dictionary
Arkansas links added to dictionary
California links added to dictionary
Colorado links added to dictionary
Connecticut links added to dictionary
Delaware links added to dictionary
District of Columbia links added to dictionary
Florida links added to dictionary
Georgia links added to dictionary
Hawaii links added to dictionary
Idaho links added to dictionary
Illinois links added to dictionary
Indiana links added to dictionary
Iowa links added to dictionary
Kansas links added to dictionary
Kentucky links added to dictionary
Louisiana links added to dictionary
Maine links added to dictionary
Maryland links added to dictionary
Massachusetts links added to dictionary
Michigan links added to dictionary
Minnesota links added to dictionary
Mississippi links added to dictionary
Missouri links added to dictionary
Montana links added to dictionary
Nebraska links added to dictionary
Nevada links ad

### State Industry Scraper 
*Please note, running this scraper will take substantial time as there are 13 tables being scraped in each state.*

#### Directory Set Up
The upcoming scraper was designed to extract the entire HTML page and save this data to an `.xls` file. In doing so, the scraper will produce 13 individual files per state. In order to maintain a clean working space, the next two cells will create directories for storing these files.

In [5]:
if not os.path.exists('./state_employment'):
    os.mkdir('./state_employment')

In [6]:
if not os.path.exists('./state_data'):
    os.mkdir('./state_data')

#### Preliminary Steps
The function `remove_special_char` was created to remove special characters (specifically, footnotes) found in the state level data. The method is vital as otherwise the data would be stored as a datatype object, and thus would not be usable in upcoming modeling.

The `subsegment` dictionary below is used to ensure that the scraper will classify and store files appropriately by industry. Through the use of Regular Expressions in the code, the url is parsed and compared to this dictionary.

Source of the subsegment dictionary can be found here: https://download.bls.gov/pub/time.series/sm/sm.supersector

In [7]:
def remove_special_char(column):
    return column.map(lambda s: re.findall('^\d+\.?\d*', str(s))[0])

In [8]:
subsegment = {'00':'Total Nonfarm', '05':'Total Private', '06':'Goods Producing',
              '07':'Service-Providing', '08':'Private Service Providing', '10':'Mining and Logging',
              '15':'Mining, Logging and Construction', '20':'Construction', '30':'Manufacturing', 
              '31':'Durable Goods', '32':'Non-Durable Goods', '40':'Trade, Transportation, and Utils', 
              '41':'Wholesale Trade', '42':'Retail Trade', '43':'Transportation and Utils', 
              '50':'Information', '55':'Financial Activities', '60':'Professional & Business Services', 
              '65':'Education & Health Services', '70':'Leisure & Hospitality', '80':'Other Services', 
              '90':'Government'}

In [9]:
state_data = {}
state_employment = {}

for state, links in list(state_backdata.items()):

    state_name = state

    dfs = {}
    employment = {}

    for i, link  in enumerate(links):
        if i == 0:
            resp = requests.get(link)
            data_name = f'./state_employment/{state_name}_Employment'
            output = open(f'{data_name}.xls', 'wb')
            output.write(resp.content)
            output.close()

            df_list = pd.read_html(f'{data_name}.xls')
            df_employment = pd.DataFrame(df_list[1])
            # remove footnote markers
            df_employment = df_employment.drop(df_employment.index[-1])
            df_target = df_employment[['labor force participation rate','employment-population ratio', 'labor force', 'employment', 'unemployment', 'unemployment rate']] # where to apply regex
            df_target = df_target.apply(remove_special_char).astype(float)
            df_employment.loc[:, df_target.columns] = df_target
            state_employment[state_name] = df_employment

            print(f'{state_name} added to state employment dictionary.')
            time.sleep(1)
            
        # pull in industry subsegment information
        else:
            resp = requests.get(link)
            sub_name = subsegment[re.findall("\/([A-Z\d\s]+)", link)[0][10:12]]
            data_name = f'./state_data/{state_name}_{sub_name}'
            output = open(f'{data_name}.xls', 'wb')
            output.write(resp.content)
            output.close()

            df_list = pd.read_html(f'{data_name}.xls')
            df = pd.DataFrame(df_list[1])
            df.drop([10, 11], inplace=True) # removing unnecessary rows
            df.set_index('Year', inplace=True)
            dfs[sub_name] = df.astype(float) # create state-subsegment entry for state dictionary
            print(f'{state_name}_{sub_name} added to list.')
            time.sleep(1)
    
    state_data[state_name] = dfs
    print(f'{state_name} data added to state data dictionary')

Alabama added to state employment dictionary.
Alabama_Total Nonfarm added to list.
Alabama_Mining and Logging added to list.
Alabama_Construction added to list.
Alabama_Manufacturing added to list.
Alabama_Trade, Transportation, and Utils added to list.
Alabama_Information added to list.
Alabama_Financial Activities added to list.
Alabama_Professional & Business Services added to list.
Alabama_Education & Health Services added to list.
Alabama_Leisure & Hospitality added to list.
Alabama_Other Services added to list.
Alabama_Government added to list.
Alabama data added to state data dictionary
Alaska added to state employment dictionary.
Alaska_Total Nonfarm added to list.
Alaska_Mining and Logging added to list.
Alaska_Construction added to list.
Alaska_Manufacturing added to list.
Alaska_Trade, Transportation, and Utils added to list.
Alaska_Information added to list.
Alaska_Financial Activities added to list.
Alaska_Professional & Business Services added to list.
Alaska_Education & 

### The Following Cells Modify the Data for 2018 

#### Compute Average by Industry
This cell calculates the yearly average value by state and industry, then saves to a dictionary.

In [10]:
target_year = '2018'

state_avgs_2018 = {}

for state_name, data in state_data.items():
    year_avgs = {}

    for subsegment, data_local in data.items():
        year_avgs[subsegment] = round(data_local.loc[target_year, :].mean(), 3)
        print(f'{state_name} {target_year} data added to year averages')

    state_avgs_2018[state_name] = year_avgs
    print(f'{state_name} averages added to state year dictionary')

Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama 2018 data added to year averages
Alabama averages added to state year dictionary
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data added to year averages
Alaska 2018 data add

#### Save to DataFrame
In this step, we save our previously modified data from a dictionary to a transposed Pandas DataFrame.

In [11]:
economies_2018 = pd.DataFrame(state_avgs_2018).T
economies_2018

,Total Nonfarm,Mining and Logging,Construction,Manufacturing,"Trade, Transportation, and Utils",Information,Financial Activities,Professional & Business Services,Education & Health Services,Leisure & Hospitality,Other Services,Government,"Mining, Logging and Construction"
Alabama,2046.258,10.000,89.200,266.992,383.467,21.092,96.258,244.242,245.167,205.933,97.117,386.792,NaN
Alaska,327.658,12.658,15.833,12.500,64.408,5.608,11.767,27.358,50.442,35.650,11.058,80.375,NaN
Arizona,2857.717,13.050,157.400,171.450,534.525,47.558,220.008,434.033,445.417,325.975,92.442,415.858,NaN
Arkansas,1267.492,5.933,50.833,160.800,253.408,12.433,61.058,145.425,191.575,118.267,55.650,212.108,NaN
California,17172.225,22.492,860.683,1323.550,3045.983,542.850,837.875,2670.217,2722.283,1993.142,571.667,2581.483,NaN
Colorado,2726.925,28.525,173.125,147.508,470.375,75.617,171.617,423.550,340.800,339.450,110.958,445.400,NaN
Connecticut,1699.275,0.567,58.767,160.667,296.500,31.667,125.467,221.092,344.792,157.783,65.617,236.358,NaN
Delaware,461.508,NaN,NaN,27.050,80.700,4.067,47.817,63.425,79.700,51.767,18.508,66.183,22.300
District of Columbia,792.958,NaN,NaN,1.342,33.208,19.158,29.592,168.117,130.892,79.808,76.783,238.400,15.650
Florida,8780.950,5.742,542.617,372.908,1779.892,139.492,575.800,1361.775,1305.058,1229.408,353.075,1115.008,NaN


#### Resolve NaN's
In this cell, we address an issue wherein seven states had a modified way of reporting industry data. In these scenarios, the `Mining and Logging` and `Construction` industries are reported as one industry, rather than being seperate (as they are for the remaining states). To resolve this, we replace the `NaN` values in those 44 states with the sum of the values from the `Mining and Logging` and `Construction` columns, then drop those columns from the DataFrame.

In [12]:
economies_2018['Mining, Logging and Construction'] = economies_2018['Mining, Logging and Construction'].fillna(economies_2018['Mining and Logging'] + economies_2018['Construction'])
economies_2018 = economies_2018.drop(columns=['Mining and Logging', 'Construction'])
economies_2018

,Total Nonfarm,Manufacturing,"Trade, Transportation, and Utils",Information,Financial Activities,Professional & Business Services,Education & Health Services,Leisure & Hospitality,Other Services,Government,"Mining, Logging and Construction"
Alabama,2046.258,266.992,383.467,21.092,96.258,244.242,245.167,205.933,97.117,386.792,99.200
Alaska,327.658,12.500,64.408,5.608,11.767,27.358,50.442,35.650,11.058,80.375,28.491
Arizona,2857.717,171.450,534.525,47.558,220.008,434.033,445.417,325.975,92.442,415.858,170.450
Arkansas,1267.492,160.800,253.408,12.433,61.058,145.425,191.575,118.267,55.650,212.108,56.766
California,17172.225,1323.550,3045.983,542.850,837.875,2670.217,2722.283,1993.142,571.667,2581.483,883.175
Colorado,2726.925,147.508,470.375,75.617,171.617,423.550,340.800,339.450,110.958,445.400,201.650
Connecticut,1699.275,160.667,296.500,31.667,125.467,221.092,344.792,157.783,65.617,236.358,59.334
Delaware,461.508,27.050,80.700,4.067,47.817,63.425,79.700,51.767,18.508,66.183,22.300
District of Columbia,792.958,1.342,33.208,19.158,29.592,168.117,130.892,79.808,76.783,238.400,15.650
Florida,8780.950,372.908,1779.892,139.492,575.800,1361.775,1305.058,1229.408,353.075,1115.008,548.359


#### Calculate Percentages
This cell calculates the percentage of each industry in relation to the `Total Nonfarm` column, and saves these percentages to a new column.

In [13]:
economies_2018_pcts = economies_2018.copy()

for column in economies_2018.columns[1:]:
    new_column = f'pct_{column}'
    economies_2018_pcts[new_column] = round(economies_2018[column] / economies_2018['Total Nonfarm'], 4) 

economies_2018_pcts.head()

,Total Nonfarm,Manufacturing,"Trade, Transportation, and Utils",Information,Financial Activities,Professional & Business Services,Education & Health Services,Leisure & Hospitality,Other Services,Government,...,pct_Manufacturing,"pct_Trade, Transportation, and Utils",pct_Information,pct_Financial Activities,pct_Professional & Business Services,pct_Education & Health Services,pct_Leisure & Hospitality,pct_Other Services,pct_Government,"pct_Mining, Logging and Construction"
Alabama,2046.258,266.992,383.467,21.092,96.258,244.242,245.167,205.933,97.117,386.792,...,0.1305,0.1874,0.0103,0.0470,0.1194,0.1198,0.1006,0.0475,0.1890,0.0485
Alaska,327.658,12.500,64.408,5.608,11.767,27.358,50.442,35.650,11.058,80.375,...,0.0381,0.1966,0.0171,0.0359,0.0835,0.1539,0.1088,0.0337,0.2453,0.0870
Arizona,2857.717,171.450,534.525,47.558,220.008,434.033,445.417,325.975,92.442,415.858,...,0.0600,0.1870,0.0166,0.0770,0.1519,0.1559,0.1141,0.0323,0.1455,0.0596
Arkansas,1267.492,160.800,253.408,12.433,61.058,145.425,191.575,118.267,55.650,212.108,...,0.1269,0.1999,0.0098,0.0482,0.1147,0.1511,0.0933,0.0439,0.1673,0.0448
California,17172.225,1323.550,3045.983,542.850,837.875,2670.217,2722.283,1993.142,571.667,2581.483,...,0.0771,0.1774,0.0316,0.0488,0.1555,0.1585,0.1161,0.0333,0.1503,0.0514


#### Save 2018 Data to Source Data Directory

In [14]:
economies_2018_pcts.to_csv('./source_data/economies_2018.csv')

### The Following Cells Modify the Data for 2021 

#### Compute Average by Industry
This cell calculates the yearly average value by state and industry, then saves to a dictionary.

In [15]:
target_year = '2021'

state_avgs_2021 = {}

for state_name, data in state_data.items():
    year_avgs = {}

    for subsegment, data_local in data.items():
        year_avgs[subsegment] = round(data_local.loc[target_year, :].mean(), 3)
        print(f'{state_name} {target_year} data added to year averages')

    state_avgs_2021[state_name] = year_avgs
    print(f'{state_name} averages added to state year dictionary')

Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama 2021 data added to year averages
Alabama averages added to state year dictionary
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data added to year averages
Alaska 2021 data add

#### Save to DataFrame
In this step, we save our previously modified data from a dictionary to a transposed Pandas DataFrame.

In [16]:
economies_2021 = pd.DataFrame(state_avgs_2021).T
economies_2021

,Total Nonfarm,Mining and Logging,Construction,Manufacturing,"Trade, Transportation, and Utils",Information,Financial Activities,Professional & Business Services,Education & Health Services,Leisure & Hospitality,Other Services,Government,"Mining, Logging and Construction"
Alabama,2039.650,8.575,94.333,263.567,394.550,19.958,98.017,250.492,239.050,188.392,95.058,387.658,NaN
Alaska,310.458,10.450,15.925,12.500,61.050,4.775,10.783,26.458,50.508,30.350,10.475,77.183,NaN
Arizona,2957.950,11.992,177.525,180.817,583.842,47.383,245.650,444.750,464.508,303.942,91.125,406.417,NaN
Arkansas,1282.608,5.300,55.233,157.183,256.342,11.708,65.892,146.117,195.100,115.800,66.958,206.975,NaN
California,16705.817,19.067,880.417,1272.492,3033.142,566.550,823.083,2703.400,2809.083,1630.642,500.008,2467.933,NaN
Colorado,2745.258,19.733,176.950,148.675,486.225,76.325,177.750,452.750,347.850,306.617,113.675,438.708,NaN
Connecticut,1614.067,0.492,59.667,153.408,290.475,29.842,117.575,213.175,333.408,133.825,58.233,223.967,NaN
Delaware,449.433,NaN,NaN,24.758,80.875,3.558,47.567,62.892,77.583,45.058,18.242,65.867,23.042
District of Columbia,742.292,NaN,NaN,1.092,29.242,19.567,28.083,167.208,120.100,49.575,71.233,241.150,15.042
Florida,8915.367,5.358,575.683,388.083,1840.283,138.550,621.783,1455.658,1340.283,1122.650,334.008,1093.083,NaN


#### Resolve NaN's
In this cell, we address an issue wherein seven states had a modified way of reporting industry data. In these scenarios, the `Mining and Logging` and `Construction` industries are reported as one industry, rather than being seperate (as they are for the remaining states). To resolve this, we replace the `NaN` values in those 44 states with the sum of the values from the `Mining and Logging` and `Construction` columns, then drop those columns from the DataFrame.

In [17]:
economies_2021['Mining, Logging and Construction'] = economies_2021['Mining, Logging and Construction'].fillna(economies_2021['Mining and Logging'] + economies_2021['Construction'])

economies_2021 = economies_2021.drop(columns=['Mining and Logging', 'Construction'])

#### Calculate Percentages
This cell calculates the percentage of each industry in relation to the `Total Nonfarm` column, and saves these percentages to a new column.

In [18]:
economies_2021_pcts = economies_2021.copy()

for column in economies_2021.columns[1:]:
    new_column = f'pct_{column}'
    economies_2021_pcts[new_column] = round(economies_2021[column] / economies_2021['Total Nonfarm'], 4) 

economies_2021_pcts.head()

,Total Nonfarm,Manufacturing,"Trade, Transportation, and Utils",Information,Financial Activities,Professional & Business Services,Education & Health Services,Leisure & Hospitality,Other Services,Government,...,pct_Manufacturing,"pct_Trade, Transportation, and Utils",pct_Information,pct_Financial Activities,pct_Professional & Business Services,pct_Education & Health Services,pct_Leisure & Hospitality,pct_Other Services,pct_Government,"pct_Mining, Logging and Construction"
Alabama,2039.650,263.567,394.550,19.958,98.017,250.492,239.050,188.392,95.058,387.658,...,0.1292,0.1934,0.0098,0.0481,0.1228,0.1172,0.0924,0.0466,0.1901,0.0505
Alaska,310.458,12.500,61.050,4.775,10.783,26.458,50.508,30.350,10.475,77.183,...,0.0403,0.1966,0.0154,0.0347,0.0852,0.1627,0.0978,0.0337,0.2486,0.0850
Arizona,2957.950,180.817,583.842,47.383,245.650,444.750,464.508,303.942,91.125,406.417,...,0.0611,0.1974,0.0160,0.0830,0.1504,0.1570,0.1028,0.0308,0.1374,0.0641
Arkansas,1282.608,157.183,256.342,11.708,65.892,146.117,195.100,115.800,66.958,206.975,...,0.1225,0.1999,0.0091,0.0514,0.1139,0.1521,0.0903,0.0522,0.1614,0.0472
California,16705.817,1272.492,3033.142,566.550,823.083,2703.400,2809.083,1630.642,500.008,2467.933,...,0.0762,0.1816,0.0339,0.0493,0.1618,0.1681,0.0976,0.0299,0.1477,0.0538


#### Save 2021 Data to Source Data Directory

In [19]:
economies_2021_pcts.to_csv('./source_data/economies_2021.csv', index_label='State')

#### Save Unemployment Data to State Employment Directory

In [20]:
for state, dataframe in state_employment.items():
    filename = f'./state_employment/{state}_employment.csv'
    dataframe.to_csv(filename, index=False)